In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.fft as sp
from scipy.signal import correlate as correlate

In [ ]:
#### Load the files here, use the [number:] if you already have a delay number to correct for
#### -n delay means [n:] for IQ0, n delay means [n:] for IQ1

IQ0 = np.fromfile("delay_test/GPS_clock_Airspy0", dtype="complex64")#[1345:]
IQ1 = np.fromfile("delay_test/GPS_clock_Airspy1", dtype="complex64")

#IQ0 = np.fromfile("Airspy0", dtype="complex64")
#IQ1 = np.fromfile("Airspy1", dtype="complex64")
#IQ2 = np.fromfile("Airspy2", dtype="complex64")


In [ ]:
#### Get the lengths of the files. These need not be equal, even after correcting for delay
#### since there also is a delay turning them off.
print(len(IQ0))
print(len(IQ1))

In [ ]:
#### This code takes a while, only run on smaller subsets, or be prepared to wait a while
#### A very good measure however to test if your delay correction is correct.
#### The print should return 0, if the correction if proper, otherwise it prints the delay
#### It will onlt print the delay if the delay is constant, flat. If there is a linear delay, solve that first.
corr = correlate(IQ0, IQ1, mode='same')
plt.plot(np.abs(corr))
plt.show()
print(len(IQ1)//2 - np.argmax(np.abs(corr)))

In [ ]:
#### This code is significantly faster, and used to track dropped samples by testing
#### coherency as function of time. Ignore the first [start] terms, then correlate
#### a subarray of length [upto], and jump through the full array at steps [step].
#### A straight line is expected. If it's not at 0, there is a full-sample delay between the two.

#### If the line is not flat, you have big problems.
#### If the line is linearly increasing, your clocks are at different frequencies, thus the GPS clock is either not connected or fucked.
#### If the line is all over the place, you are dropping samples and your data is useless.

step  = 10000
upto  = 20000         # good value is 2x step, to make sure you aren't skipping data.
start = 20000
end   = 140_000_000   # make sure this is less than your total arraysize.

arr = []
for s in np.arange(start, end, step).astype('int'):
    c = correlate(IQ0[s:s+upto], IQ1[s:s+upto], mode='same')
    shift = upto // 2 - np.argmax(np.abs(c)) 
    arr += [shift]
plt.plot(arr)
plt.show()

In [ ]:
#### This code is preparational code for the 2D crosscorrelation.
#### It makes sure the lengths are multiples of the channel count, and that the arrays are equal length.

channels = 512
sample_rate = 2.5e6

#### changing length to a multiple of channels
len0 = (len(IQ0)//channels) * channels
len1 = (len(IQ1)//channels) * channels
shortest = np.min((len0, len1))

IQ0 = IQ0[:shortest]
IQ1 = IQ1[:shortest]

l = len(IQ0) / channels
time_len = len(IQ0) / sample_rate
print("Size of array:", len(IQ0), "\nLength of observation in seconds:", time_len)

In [ ]:
#### This cell reshapes the time-series data into channels.
#### The 'W' is optional, and can be multiplied with your IQ0 / IQ1 arrays after reshaping.
#### It represents the blackman-harris windowing. Only for debugging atm, without is fine.

#w =  np.blackman(channels).astype('float32')
IQ0 = (IQ0.reshape((-1, channels))).astype('complex64') #*w
IQ1 = (IQ1.reshape((-1, channels))).astype('complex64') #*w

In [ ]:
#### This is the actual FFT. Numpy didnt like being used, so its the Scipy one.
#### Division by l is for normalization, and is optional.

IQ0 = sp.fftshift(sp.fft2(IQ0)) / l
IQ1 = sp.fftshift(sp.fft2(IQ1)) / l

In [ ]:
#### This calculates the crosscorrelation, both the phase and magnitude.
#### If you only need one for testing, comment the other to save ram and cpu time.
#### It can also calculate the autocorrelation if need be.

cross_ang = np.angle(IQ0 * np.conj(IQ1))
cross_abs = np.abs(IQ0 * np.conj(IQ1))

#auto = np.abs(IQ0**2)

In [ ]:
#### Plotting the 2D result.
#### Freq is the frequency range in MHz, ext and asp are parameters for the plt.imshow plotting, change asp
#### if the plot is weirdly shaped.

#### Expected result is a phase plot that is stable in time, and linear as function of frequency. 
#### If this is not the case, and its chaos or more regions than [-1, 0, 1] in terms of phase as function of frequency,
#### then your delay is incorrect. Chaos means its fucked, more than 3 regions means there is likely a sub-sample delay
#### which the earlier sub-array correlator cannot see.

freq = [1420 - 1.25, 1420 + 1.25]
ext = [freq[0], freq[1], 0, time_len]
asp = IQ0.shape[0] / time_len * (freq[1] - freq[0]) / channels / 256

fig, (ax1, ax2) = plt.subplots(1, 2, sharey=True)
p1 = ax1.imshow(cross_ang, aspect=asp, vmin=-np.pi, vmax=np.pi, cmap='seismic', extent=ext, origin='lower')#, interpolation='none')
p2 = ax2.imshow(cross_abs, aspect=asp, extent=ext, vmax=2*np.mean(cross_abs), origin='lower')
plt.colorbar(p1, ax=ax1)
plt.colorbar(p2, ax=ax2)
ax1.set_xlabel("Frequency (MHz)")
ax2.set_xlabel("Frequency (MHz)")
ax1.set_ylabel("Time (s)")
ax1.set_title("Phase")
ax2.set_title("Magnitude")
plt.suptitle("Phase and magnitude of the crosscorrelation\nof the shifted resistor observation")
plt.tight_layout()
plt.show()

In [ ]:
#### The sum of the phase over the time-axis can also be useful to see slight deviations.
#### It is plotted below.
#### If the phase plot is not a linear line going from -1 to 0 to 1, something is going wrong.
#### If the phase is chaos, its fucked, and if its smooth but not linear, then there is a sub-sample delay.

fig, (ax1, ax2) = plt.subplots(1, 2)
x = np.linspace(freq[0], freq[1], channels)
ax1.plot(x, cross_ang.mean(axis=0))
ax2.plot(x, cross_abs.mean(axis=0))
ax1.set_xlabel("Frequency (MHz)")
ax2.set_xlabel("Frequency (MHz)")
ax1.set_ylabel("Phase")
ax2.set_ylabel("Magnitude")
ax1.set_title("Phase")
ax2.set_title("Magnitude")
ax1.tick_params(rotation=30)
ax2.tick_params(rotation=30)
plt.suptitle("Mean phase and magnitude of the crosscorrelation\nof the shifted resistor observation")
plt.tight_layout()
plt.show()

In [ ]:
#### Code for sub-sample delay correction based on phase goes here.
#### (the code is done, but its not here yet, ask Floris if you need it)